In [ ]:
pip install git+https://github.com/tky823/Audyn.git pyloudnorm

In [1]:
import torch
import torchaudio
import pyloudnorm as pyln

In [2]:
from audyn.utils.data.download import download_file
from audyn.functional.loudness import normalize_by_loudness

In [3]:
url = "https://github.com/tky823/ssspy-data/raw/refs/heads/main/audio/monoral_16k_5sec.wav"
path = "monoral_16k_5sec.wav"
download_file(url, path)

Download file to monoral_16k_5sec.wav: 100%|██████████| 160k/160k [00:00<00:00, 3.43MB/s]


In [4]:
waveform, sample_rate = torchaudio.load(path)
waveform = waveform.to(torch.float64)
waveform_by_pyln = waveform.permute(1, 0).numpy()

In [5]:
loudness = -12

In [6]:
def normalize_by_pyln_loudness(waveform: torch.Tensor, sample_rate: int, loudness: float) -> torch.Tensor:
    _waveform = waveform.permute(1, 0).numpy()
    meter = pyln.Meter(sample_rate)
    _loudness = meter.integrated_loudness(_waveform)
    normalized_waveform = pyln.normalize.loudness(_waveform, _loudness, loudness)
    normalized_waveform = torch.from_numpy(normalized_waveform)
    normalized_waveform = normalized_waveform.permute(1, 0)

    return normalized_waveform

In [7]:
normalized_waveform = normalize_by_loudness(waveform, sample_rate=sample_rate, loudness=loudness)
normalized_waveform_pyln = normalize_by_pyln_loudness(waveform, sample_rate=sample_rate, loudness=loudness)

/usr/local/lib/python3.11/dist-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


In [8]:
torch.allclose(normalized_waveform, normalized_waveform_pyln)

True